<a href="https://colab.research.google.com/github/ThunchanokChai/SC637703-2023/blob/main/individual_work.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

ข้อมูลที่ใช้ คือ ข้อมูลการจำแนกข้าว
ซึ่งประกอบไปด้วย 12 ตัวแปร ดังนี้
*   id
*   Area
*   MajorAxisLength
*   MinorAxisLength
*   Eccentricity
*   ConvexArea
*   EquivDiameter
*   Extent
*   Perimeter
*   Roundness
*   AspectRation
*   Class โดยที่ Jasmine - 1, Gonen - 0.

อ้างอิง : https://www.kaggle.com/datasets/mssmartypants/rice-type-classification

#import data

In [ ]:
rice = read.csv("/content/riceClassification.csv")
head(rice)

#Data Explore

In [ ]:
install.packages('DataExplorer')

In [ ]:
library('DataExplorer')
head(rice)

In [ ]:
summary(rice)

In [ ]:
introduce(rice)

#Select Data


In [ ]:
rice_new = rice[,2:12] #ืลบ Column ที่ 1 ออกเนื่องจากไม่ได้ใช้ในการวิเคราะห์

In [ ]:
head(rice_new)

#PCA Data

##Split data into X and Y

In [ ]:
X = rice_new[,1:10]
head(X)

In [ ]:
Y = rice_new[,11]
head(Y)

##Check the Correlation

In [ ]:
pairs(X, pch = 19, lower.panel = NULL)

##PCA

In [ ]:
install.packages("factoextra")

In [ ]:
library("ggplot2")
library("factoextra")

In [ ]:
res.pca = prcomp(X,scale = TRUE)

In [ ]:
print(summary(res.pca))
#เลือก 2 PC เนื่องจาก 2 PC Cumulative proportion > 0.8

In [ ]:
Sd_PC = res.pca$sdev
eigval = Sd_PC^2
eigvec = res.pca$rotation
eigval_mean = mean(eigval)
#แสดงค่าeigenvalue
eigval

In [ ]:
#มี PC1 และ PC2 ที่ eigenvalue > 1

In [ ]:
var = get_pca_var(res.pca)

###Scree plot

In [ ]:
fviz_pca_var(res.pca,col.var="cos2", gradient.cols=c("#A1D0BE","#F68D71","#90A9D7"))

##Correlation circle

In [ ]:
library(corrplot)

In [ ]:
corrplot(var$cos2,method = "color", type = "full",  diag = TRUE,tl.col = "black", bg ="white", col = NULL)

###create new data frame

In [ ]:
# New Data
PCA = res.pca$x[,1:2]

In [ ]:
# สร้าง df ใหม่ที่รวม X_train และ Y_train
PCA_X_data <- cbind(PCA,Y)

In [ ]:
# เปลี่ยนชื่อ Y_train เป็น Class
colnames(PCA_X_data)[colnames(PCA_X_data)=="Y"] = "Class"

In [ ]:
head(PCA_X_data)

In [ ]:
# ทำให้เป็นdata frame จะได้ data ใหม่
PCA_data = as.data.frame(PCA_X_data)

#Prepare Data before ML

### Split normal data into training (80%) and test (20%) data

In [ ]:
install.packages("caret")

In [ ]:
library(caret)
library(dplyr)

In [ ]:
set.seed(555)
training.samples = rice_new$Class %>% createDataPartition(p = 0.8, list = FALSE)

In [ ]:
train_data = rice_new[training.samples, ]
head(train_data)

In [ ]:
test_data = rice_new[-training.samples, ]
head(test_data)

### Split PCA data into training (80%) and test (20%) data

In [ ]:
set.seed(555)
PCA_training_samples = PCA_data$Class %>% createDataPartition(p = 0.8, list = FALSE)

In [ ]:
PCA_train_data = PCA_data[PCA_training_samples, ]
head(PCA_train_data)

In [ ]:
PCA_test_data = PCA_data[-PCA_training_samples, ]
head(PCA_test_data)


#Quadratic Discriminant Analysis(QDA) without PCA

##Fit the QDA Model

In [ ]:
install.packages("MASS")

In [ ]:
library(MASS)
model = qda(Class~., data=train_data)

In [ ]:
model

##use QDA model to make predictions on test data

In [ ]:
predicted = predict(model, test_data)

In [ ]:
names(predicted)

In [ ]:
head(predicted$x)

In [ ]:
head(predicted$class)

##find accuracy of model

In [ ]:
mean(predicted$class==test_data$Class)

##Visualize the Results

In [ ]:
library(ggplot2)
ggplot(test_data, aes(Area,MajorAxisLength)) +
  geom_point(aes(color = Class))

#Quadratic Discriminant Analysis(QDA) with PCA

##Fit the QDA Model

In [ ]:
model = qda(Class~., PCA_train_data)

In [ ]:
model

##use QDA model to make predictions on test data

In [ ]:
predicted = predict(model, PCA_test_data)

In [ ]:
names(predicted)

In [ ]:
head(predicted$x)

In [ ]:
head(predicted$class)

##find accuracy of model

In [ ]:
mean(predicted$class==PCA_test_data$Class)

##Visualize the Results

In [ ]:
ggplot(PCA_test_data, aes(PC1,PC2)) +
  geom_point(aes(color = Class))

#COMPARE QDA with and without PCA

In [ ]:
#accuracy of QDA model = 0.988452020896343

In [ ]:
#accuracy of QDA model with PCA = 0.98625240582898

In [ ]:
#####QDA model with PCA better than QDA model

#Logistic Regression without PCA

##Fit the logistic model

###LOGISTIC REGRESSION - FULL MODEL

In [ ]:
full_model = glm(Class ~., data= train_data, family = binomial)
summary(full_model)

###LOGISTIC REGRESSION - STEPWISE MODEL

In [ ]:
stepwise_model = glm(Class ~., data= train_data, family = binomial) %>% stepAIC(trace = FALSE)
summary(stepwise_model)

###COMPARE FULL and STEPWISE MODEL

####Full Model

In [ ]:
# Prediction of Probability
prob_full = predict(full_model, test_data, type = "response")
head(prob_full)

In [ ]:
# Predicted Class
predicted_full = ifelse(prob_full > 0.5, "1", "0")
head(predicted_full)

In [ ]:
# Accuracy
mean(predicted_full == test_data$Class)

####Stepwise Model

In [ ]:
# Prediction of Probability
prob_stepwise = predict(stepwise_model, test_data, type = "response")
head(prob_stepwise)

In [ ]:
# Predicted Class
predicted_stepwise = ifelse(prob_stepwise > 0.5, "1", "0")
head(predicted_stepwise)


In [ ]:
# Accuracy
mean(predicted_stepwise == test_data$Class)

In [ ]:
####STEPWISE MODEL is better than FULL MODEL

##Goodness of Fit of STEPWISE MODEL

In [ ]:
install.packages("performance")

In [ ]:
library(performance)
performance_hosmer(stepwise_model, n_bins = 10)

##Assumption and Diagnostic Checking

###Linearity

In [ ]:
library(tidyverse)
library(ggplot2)
library(broom)
mydata = train_data %>% dplyr :: select_if(is.numeric)

probabilities_stepwise = predict(stepwise_model, train_data, type = "response")
logit_sw = log(probabilities_stepwise/(1 - probabilities_stepwise))
mydata2 = mydata %>% mutate(logit_sw) %>% gather(key = "predictors", value = "predictors.value",-logit_sw)

head(mydata2,10)

ggplot(mydata2, aes(logit_sw,predictors.value)) + geom_point(size = 0.5, alpha = 0.5) + geom_smooth(method="loess") + theme_bw() + facet_wrap(~predictors,scales="free_y")

###Influential Values

In [ ]:
model_sw_data = augment(stepwise_model) %>% mutate(index=1:n())
model_sw_data %>% top_n(3,.cooksd)
ggplot(model_sw_data,aes(index,.std.resid)) + geom_point(aes(color = Class),alpha = 5) + theme_bw()

###Multicolinearity

In [ ]:
install.packages("car")

In [ ]:
library(car)
vif(stepwise_model)

##ODD RATIO

In [ ]:
model.OR <- exp(stepwise_model$coefficients)
round(model.OR, 3)

#Logistic Regression with PCA

##Fit the logistic model

###LOGISTIC REGRESSION - FULL MODEL

In [ ]:
full_model = glm(Class ~., data = PCA_train_data, family = binomial)
summary(full_model)

###use logistic regression model to make predictions on test data

In [ ]:
# Prediction of Probability
prob_full = predict(full_model, PCA_test_data , type = "response")
head(prob_full)

In [ ]:
# Predicted Class
predicted_full = ifelse(prob_full > 0.5, "1", "0")
head(predicted_full)


##Find accuracy of model

In [ ]:
# Accuracy
mean(predicted_full == PCA_test_data$Class)

##Goodness of Fit

In [ ]:
library(performance)
performance_hosmer(full_model, n_bins = 4)

##Assumption and Diagnostic Checking

###Linearity

In [ ]:
mydata = PCA_train_data %>% dplyr :: select_if(is.numeric)

probabilities_stepwise = predict(stepwise_model, PCA_train_data, type = "response")
logit_sw = log(probabilities_stepwise/(1 - probabilities_stepwise))
mydata2 = mydata %>% mutate(logit_sw) %>% gather(key = "predictors", value = "predictors.value",-logit_sw)

head(mydata2,10)

In [ ]:
ggplot(mydata2, aes(logit_sw,predictors.value)) + geom_point(size = 0.5, alpha = 0.5) + geom_smooth(method="loess") + theme_bw() + facet_wrap(~predictors,scales="free_y")

###Influential Values

In [ ]:
model_sw_data = augment(stepwise_model) %>% mutate(index=1:n())

model_sw_data %>% top_n(3,.cooksd)

ggplot(model_sw_data,aes(index,.std.resid)) + geom_point(aes(color = Class),alpha = 5) + theme_bw()


###Multicolinearity

In [ ]:
library(car)
vif(stepwise_model)

##ODD RATIO

In [ ]:
model.OR <- exp(stepwise_model$coefficients)
round(model.OR, 3)

#Compare logistic model with and without PCA

In [ ]:
#accuracy of Logistic model = 0.991201539730547

In [ ]:
#accuracy of Logistic model with PCA = 0.989001924663184

In [ ]:
#####Logistic model better than Logistic model with PCA